In [ ]:
using LinearAlgebra
using JuMP
using GLPK
using Plots
pyplot()

In [ ]:
# A função "PLI" abaixo recebe 10 parâmteros.
# Modo de chamar: "PLI(max_min, i, j, weights, A, b, C, d, E, f)"

# Parâmetros:
# max_min := 1 se for um problema de maximização, 0 se for um problema de minimização.
# i, j := dimensões do vetor de variáveis (se for unidimensional, j=1).
# weights := vetor de pesos da função objetivo (em ordem matricial).
# A, b := matriz e vetor das restrições de igualdade.
# C, d := matriz e vetor das restrições de desigualdade <=.
# E, f := matriz e vetor das restrições de desigualdade >=.

# Observação: Restrições que não se aplicarem ao problema devem ser 0 ao chamar a função.

# A função printa 3 resultados: Função objetivo, valor ótimo e solução ótima.

In [207]:
function PLI(max_min, i, j, weights, A, b, C, d, E, f)
    model = Model(GLPK.Optimizer) # Cria modelo
    p = length(weights)

    # Define variáveis
    @variable(model, x[1:i, 1:j] >= 0)
    x = reshape(x', p, 1)

    # Define restrições
    for l in 1:length(f)
        @constraint(model, (E*x)[l] >= f[l]) # Restrições de desigualdade >=
    end
    for m in 1:length(b)
        @constraint(model, (A*x)[m] == b[m]) # Restrições de igualdade
    end
    for n in 1:length(d)
        @constraint(model, (C*x)[n] <= d[n]) # Restrições de desigualdade <=
    end

    # Define função objetivo
    if max_min == 1
        @objective(model, Max, sum(weights[k] * x[k] for k in 1:p)) # Problema de maximização
    else
        @objective(model, Min, sum(weights[k] * x[k] for k in 1:p)) # Problema de minimização
    end
    
    # Roda a otimização
    optimize!(model)
    
    # Exibe resultados
    println("Função objetivo: ", objective_function(model))
    println()
    println("Valor ótimo: ", objective_value(model))
    println()
    println("Solução ótima:")
    for k in 1:p
        println(x[k], " = ", value(x[k]))
    end
end

PLI (generic function with 3 methods)

In [213]:
# Exemplo: Problema das usinas da fábrica (Q1-L1)
E = [1 0 0 1 0 0;0 1 0 0 1 0;0 0 1 0 0 1]
f = [100;200;300]
C = [1 1 1 0 0 0;0 0 0 1 1 1]
d = [400;300]
weights = [1 1.5 3.5 2 1 2]
i = 2
j = 3

PLI(0, i, j, weights, 0, 0, C, d, E, f)

Função objetivo: x[1,1] + 1.5 x[1,2] + 3.5 x[1,3] + 2 x[2,1] + x[2,2] + 2 x[2,3]

Valor ótimo: 1000.0

Solução ótima:
x[1,1] = 100.0
x[1,2] = 200.0
x[1,3] = 0.0
x[2,1] = 0.0
x[2,2] = 0.0
x[2,3] = 300.0
